## Python Module: Reserve a Network 

This notebook demonstrates how to use the OpenStack python API to reserve a simple node.
Chameleon requires most resources to be reserved before they can be used. Reservations can not be created independently. Instead, all reservatons must be part of a *Lease*.  Leases are composed of a group of one or more reservations that, typically, include all reservations required for an experiment.  

#### Related Modules

- [Reserve Server](../../modules-python/reservations/reserve_server.ipynb)
- [Get Lease](../../modules-python/reservations/get_lease_by_name.ipynb)
- [Get Reservation](../../modules-python/reservations/get_reservation_from_lease.ipynb)

#### Import Library

```
import chi
from chi.reservation_api_examples import *
```

#### Arguments

Required:

- `lease_name`: The desired name for the new lease
- `network_name`: The desired name for the new network

Optional:

- `of_controller_ip`: OpenFlow controller IP. Only required for OpenFlow networks. Default: None
- `of_controller_port`: OpenFlow controller network port. Only required for OpenFlow networks. Default: None
- `vswitch_name`: The name of the vswtich to attach additional VLANs to the a single swtich. Default: None
- `provider`: OpenStack provider network. Stitched networks need to specifiy the appopriate stitching provider (i.e. 'exogeni'): Default: 'physnet1'


#### Imported Methods used by Example(s) 

In [ ]:
def add_network_reservation(reservation_list, network_name,
                            network_description="", 
                            of_controller_ip=None, 
                            of_controller_port=None, 
                            vswitch_name=None, 
                            physical_network="physnet1"):

    if of_controller_ip != None and of_controller_port != None:
        description = description + 'OFController=' + of_controller_ip + ':' + of_controller_port 
        
    if vswitch_name != None and of_controller_ip != None and of_controller_port != None:
        description = description + ','
    
    if vswitch_name != None:
        description = description + 'VSwitchName=' + vswitch_name

    reservation_list.append({
        "resource_type": "network",
        "network_name": network_name,
        "network_description": network_description,
        "resource_properties": json.dumps(["==", "$physical_network", physical_network]),
        "network_properties": ""
    })

def reserve_network(lease_name,
                    network_name,
                    of_controller_ip=None, 
                    of_controller_port=None, 
                    vswitch_name=None, 
                    physical_network="physnet1"):
    # Set start/end date for lease
    # Start one minute into future to avoid Blazar thinking lease is in past
    # due to rounding to closest minute.
    start_date = (datetime.now(tz=tz.tzutc()) + timedelta(minutes=1)).strftime(BLAZAR_TIME_FORMAT)
    end_date   = (datetime.now(tz=tz.tzutc()) + timedelta(days=1)).strftime(BLAZAR_TIME_FORMAT)
    
    # Build list of reservations (in this case there is only one reservation)
    reservation_list = []
    add_network_reservation(reservation_list, 
                            network_name=network_name,
                            of_controller_ip=of_controller_ip, 
                            of_controller_port=of_controller_port, 
                            vswitch_name=vswitch_name, 
                            physical_network=physical_network)
    
    # Create the lease
    lease = chi.blazar().lease.create(name=lease_name, 
                                start=start_date,
                                end=end_date,
                                reservations=reservation_list, events=[])
    

## Examples: 
**Important**: Examples below import API methods from chi library and does not use refernece code above!

#### Reserve a Network

In [ ]:
import json
import os
import chi

from chi.reservation_api_examples import *

#Config with your project and site
chi.set('project_name', 'CH-816532') # Replace with your project name
chi.set('region_name', 'CHI@UC')     # Optional, defaults to 'CHI@UC'


#Get a network by name
lease = reserve_network("myLeaseName", network_name=network_name)
    
#Get the lease by name
lease = get_lease_by_name("myLeaseName")
    
#Print the lease info
print(json.dumps(lease, indent=2))


